In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import pandas as pd
import numpy as np
import warnings
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import TruncatedSVD,PCA
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from gensim.models import Word2Vec, FastText
from gensim import corpora,models
from sklearn import preprocessing
from tqdm import tqdm
import multiprocessing
warnings.filterwarnings('ignore')
print(multiprocessing.cpu_count())
import gc
from category_encoders import *
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore')
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
## You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
origin_features = []
drop_list=[]
# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

4
/kaggle/input/financial-risk/testb_op.csv
/kaggle/input/financial-risk/.doc
/kaggle/input/financial-risk/submit_example.csv
/kaggle/input/financial-risk/train_trans.csv
/kaggle/input/financial-risk/test_a_op.csv
/kaggle/input/financial-risk/test_a_base.csv
/kaggle/input/financial-risk/train_base.csv
/kaggle/input/financial-risk/train_label.csv
/kaggle/input/financial-risk/testb_base.csv
/kaggle/input/financial-risk/test_a_trans.csv
/kaggle/input/financial-risk/testb_trans.csv
/kaggle/input/financial-risk/train_op.csv


In [2]:
def encode_LE(data,test):
    for f in tqdm([f for f in data.select_dtypes('object').columns if f not in ['user','label']]):
        le = LabelEncoder()
        
        le.fit(list(data[f].values) + list(test[f].values))
        data[f] = le.transform(list(data[f].values))
        test[f] = le.transform(list(test[f].values))
        
    return data,test
        
def encode_CB(df,col1_list,col2_list):
    uids=[]
    for col1 in col1_list:
        for col2 in col2_list:
            name = col1+'_'+col2
            uids.append(name)
            df[name] = df[col1].astype(str)+'_'+df[col2].astype(str)
            df[name+'_FE'] = df[name].map(df[name].value_counts(dropna=True,normalize=True))
            df[name+'_FE'] = df[name+"_FE"].astype('float32')
    return df,uids

def encode_Cat(train,test, k,sigma=2.0,a=2.0):
    
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)    
    
    print("Cat encoding")
    for col in tqdm([f for f in data.select_dtypes('object').columns if f not in ['user','label']]):
        train['catboost_'+col] = 0
        test['catboost_' + col] =0
        if col not in origin_features:
            sigma=10
            a=20
        for trn_idx,val_idx in kf.split(train,train['label']):
            X = train.iloc[trn_idx][col]
            enc_cat = CatBoostEncoder(sigma=sigma).fit(X,train.iloc[trn_idx]['label'])
            train['catboost_'+col] += enc_cat.transform(train[col])[col] / k
            test['catboost_'+col] += enc_cat.transform(test[col])[col] / k
    print('Cat encoding finish')
    return  train,test
            

In [3]:
%%time
def gen_user_status_features(df,index,value,aggregations=['mean','std','min','max','sum']):
        
    group_df = df.groupby([index])[value].agg(aggregations).reset_index() 
    if('mean' in aggregations and 'std' in aggregations):
        group_df['CV']=group_df['std'] / group_df['mean']
    group_df.rename(columns={i:"{}_{}_".format(index,value)+i for i in group_df.columns if i not in[index]},
                   inplace=True
                   )
                
    return group_df


def gen_user_group_features(df, col,value,aggregations=['mean','std','min','max','sum']):  
    group_df = df.pivot_table(index='user',
                              columns=col,
                              values=value,
                              dropna=False,
                              aggfunc=aggregations).fillna(0)
    cols=group_df.columns
    group_df.columns = ['user_group_{}_{}_{}_{}'.format(col,value, f[1], f[0]) for f in group_df.columns]
    for f in cols:
        group_df['user_group_{}_{}_{}_CV'.format(col,value,f[1])] = group_df['user_group_{}_{}_{}_std'.format(col,value,f[1])] / group_df['user_group_{}_{}_{}_mean'.format(col,value,f[1])]
    group_df.reset_index(inplace=True)
    return group_df

def gen_user_group_nunique_features(df, col,value):
    
    group_df = df.pivot_table(index='user',
                              columns=col,
                              values=value,
                              dropna=False,
                              aggfunc=['count','nunique']).fillna(0)
    group_df.columns = ['user_{}_{}_{}_{}'.format(col,value, f[1], f[0]) for f in group_df.columns]
    group_df.reset_index(inplace=True)
    return group_df

def gen_user_nunique_features(df, value, prefix):
    a="user_{}_{}_nuniq".format(prefix, value)
    b="user_{}_{}_count".format(prefix, value)
    group_df = df.groupby(['user'])[value].agg(
        a='nunique',b='count'
    ).reset_index()
    
    group_df.rename(columns={'a':a,'b':b},inplace=True) 
    return group_df

def gen_user_null_features(df, value, prefix):
    df['is_null'] = 0
    df.loc[df[value].isnull(), 'is_null'] = 1

    group_df = df.groupby(['user'])['is_null'].agg(a='sum',
                                                    b='mean').reset_index()
    
    group_df.rename(columns={'a':'user_{}_{}_null_cnt'.format(prefix, value),
                    'b':'user_{}_{}_null_ratio'.format(prefix, value)},
                   inplace=True
                   )

    return group_df

    
def gen_user_window_amount_features(df, window):
    
    group_df = df[df['days_diff']>window].groupby('user')['amount'].agg(
        a='mean',
         b='std',
         c='max',
         d='min',
         e='sum',
        f='median',
         g='count',
        ).reset_index()
    
    group_df.rename(columns={'a':'user_amount_mean_{}d'.format(window),
                    'b':'user_amount_std_{}d'.format(window),
                    'c':'user_amount_max_{}d'.format(window),
                    'd':'user_amount_min_{}d'.format(window),
                    'e':'user_amount_sum_{}d'.format(window),
                    'f':'user_amount_med_{}d'.format(window),
                    'g':'user_amount_cnt_{}d'.format(window)},
                   inplace=True
                   )
    
    return group_df

def gen_user_window_hour_amount_features(df, window):
    
    group_df = df[df['hour']>window].groupby('user')['amount'].agg(
        a='mean',
         b='std',
         c='max',
         d='min',
         e='sum',
         f='median',
         g='count',
        ).reset_index()
    
    group_df.rename(columns={'a':'user_amount_mean_{}h'.format(window),
                    'b':'user_amount_std_{}h'.format(window),
                    'c':'user_amount_max_{}h'.format(window),
                    'd':'user_amount_min_{}h'.format(window),
                    'e':'user_amount_sum_{}h'.format(window),
                    'f':'user_amount_med_{}h'.format(window),
                    'g':'user_amount_cnt_{}h'.format(window)},
                   inplace=True
                   )
    
    return group_df


def gen_user_tfidf_features(df, value):
    #填充缺失值
    df[value].replace(' ', np.nan, inplace=True)
    df[value] = df[value].astype(str)
    df[value].fillna('-1', inplace=True)

    #
    group_df = df.groupby(['user']).apply(lambda x: x[value].tolist()).reset_index()
    group_df.columns = ['user', 'list']
    group_df['list'] = group_df['list'].apply(lambda x: ','.join(x))
    enc_vec = TfidfVectorizer()
    tfidf_vec = enc_vec.fit_transform(group_df['list'])
    svd_enc = TruncatedSVD(n_components=10, n_iter=20, random_state=2020)
    vec_svd = svd_enc.fit_transform(tfidf_vec)
    vec_svd = pd.DataFrame(vec_svd)
    vec_svd.columns = ['svd_tfidf_{}_{}'.format(value, i) for i in range(10)]
    group_df = pd.concat([group_df, vec_svd], axis=1)
    del group_df['list']
    return group_df

def gen_user_countvec_features(df, value):
    df[value] = df[value].astype(str)
    df[value].fillna('-1', inplace=True)
    group_df = df.groupby(['user']).apply(lambda x: x[value].tolist()).reset_index()
    group_df.columns = ['user', 'list']
    group_df['list'] = group_df['list'].apply(lambda x: ','.join(x))
    enc_vec = CountVectorizer()
    tfidf_vec = enc_vec.fit_transform(group_df['list'])
    svd_enc = TruncatedSVD(n_components=5, n_iter=20, random_state=2020)
    vec_svd = svd_enc.fit_transform(tfidf_vec)
    vec_svd = pd.DataFrame(vec_svd)
    vec_svd.columns = ['svd_countvec_{}_{}'.format(value, i) for i in range(5)]
    group_df = pd.concat([group_df, vec_svd], axis=1)
    del group_df['list']
    return group_df

#定义加载函数
def load_dataset(DATA_PATH):
    train_label = pd.read_csv(DATA_PATH+'train_label.csv')
    test_base = pd.read_csv(DATA_PATH+'testb_base.csv')
    
    train_base = pd.read_csv(DATA_PATH+'train_base.csv')
    train_op = pd.read_csv(DATA_PATH+'train_op.csv')
    train_trans = pd.read_csv(DATA_PATH+'train_trans.csv')
    test_op = pd.read_csv(DATA_PATH+'testb_op.csv')
    test_trans = pd.read_csv(DATA_PATH+'testb_trans.csv')
    
    origin_features = list(train_base.columns)+list(train_trans.columns)+list(train_op.columns)
    return train_label, train_base, test_base, train_op, train_trans, test_op, test_trans

#tran_trans和train_op文件中的tm_diff时间转换
def transform_time(x):
    day = int(x.split(' ')[0])
    hour = int(x.split(' ')[2].split('.')[0].split(':')[0])
    minute = int(x.split(' ')[2].split('.')[0].split(':')[1])
    second = int(x.split(' ')[2].split('.')[0].split(':')[2])
    return 86400*day+3600*hour+60*minute+second



#讲所有的数据进行初步的整合，并且把时间days_diff进行了数字化提取
def data_preprocess(DATA_PATH):
    train_label, train_base, test_base, train_op, train_trans, test_op, test_trans = load_dataset(DATA_PATH=DATA_PATH)
    # 拼接数据
    train_df = train_base.copy()
    test_df = test_base.copy()
        #将train_base数据和train_label整合
    train_df = train_label.merge(train_df, on=['user'], how='left')
    del train_base, test_base
        #将train_op数据和test_op整合--方便将train和test中所有的数据进行统一的处理
    op_df = pd.concat([train_op, test_op], axis=0, ignore_index=True)
        #将train_trans数据和test_trans整合--方便将train和test中所有的数据进行统一的处理
    trans_df = pd.concat([train_trans, test_trans], axis=0, ignore_index=True)
        #将train_base数据和test_base整合
    data = pd.concat([train_df, test_df], axis=0, ignore_index=True) 
    del train_op, test_op, train_df, test_df
    
    
    # 时间维度的处理
    op_df['days_diff'] = op_df['tm_diff'].apply(lambda x: int(x.split(' ')[0]))
    trans_df['days_diff'] = trans_df['tm_diff'].apply(lambda x: int(x.split(' ')[0]))
    
    op_df['timestamp'] = op_df['tm_diff'].apply(lambda x: transform_time(x))
    trans_df['timestamp'] = trans_df['tm_diff'].apply(lambda x: transform_time(x))
    op_df['hour'] = op_df['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[0]))
    trans_df['hour'] = trans_df['tm_diff'].apply(lambda x: int(x.split(' ')[2].split('.')[0].split(':')[0]))
    trans_df['week'] = trans_df['days_diff'].apply(lambda x: x % 7)
    
    
    # 排序
    trans_df = trans_df.sort_values(by=['user', 'timestamp'])
    op_df = op_df.sort_values(by=['user', 'timestamp'])
    trans_df.reset_index(inplace=True, drop=True)
    op_df.reset_index(inplace=True, drop=True)

    gc.collect()
    return data, op_df, trans_df

def gen_features(df_, op, trans):
    df=df_
    # base数据处理
    print("base logistic features")
    df['card_a_cnt_avg'] = df["card_a_cnt"]/df["using_time"]
    df['product7_fail_ratio'] = df['product7_fail_cnt'] / df['product7_cnt']
    
    df['city_count'] = df.groupby(['city'])['user'].transform('count')  ## 与原始数据集相同数量的项目
    df['city_product7_fail_ratio'] =  df.groupby(['city'])['product7_fail_cnt'].transform('sum')/df.groupby(['city'])['product7_cnt'].transform('sum')
    df['province_count'] = df.groupby(['province'])['user'].transform('count')
    df['province_product7_fail_ratio'] = df.groupby(['province'])['product7_fail_cnt'].transform('sum')/df.groupby(['province'])['product7_cnt'].transform('sum')
    
    df['login_cnt_period_var']=df[['login_cnt_period1','login_cnt_period2']].std(axis=1)
    df['login_cnt_period_CV']=df['login_cnt_period_var'] / df[['login_cnt_period1','login_cnt_period2']].mean(axis=1)
    df['card_var']=df[['card_a_cnt','card_b_cnt','card_c_cnt','card_d_cnt']].std(axis=1)
    df['card_CV']=df['card_var'] / df[['card_a_cnt','card_b_cnt','card_c_cnt','card_d_cnt']].mean(axis=1)
    df['op_cnt_var']=df[['op1_cnt','op2_cnt']].std(axis=1)
    df['op_cnt_CV']=df['op_cnt_var'] / df[['op1_cnt','op2_cnt']].mean(axis=1)
    df['login_var']=df[['login_cnt_period1','login_cnt_period2','login_cnt_avg']].std(axis=1)
    df['login_CV']=df['login_var'] / df[['login_cnt_period1','login_cnt_period2','login_cnt_avg']].mean(axis=1)
    
    
    df['ip_cnt_per_login_cnt_avg'] = df['ip_cnt']/df[['login_cnt_period1','login_cnt_period2','login_cnt_avg']].mean(axis=1)
    df['ip_cnt_per_login_days_cnt'] = df['ip_cnt']/df['login_days_cnt']
    df['acc_count_per_time'] = df["acc_count"]/df["using_time"]
    df['ip_cnt_per_using_time'] = df["ip_cnt"]/df["using_time"]
    df['agreement_total_per_using_time'] = df["agreement_total"]/df["using_time"]
    df['agreement_total_per_account_cnt'] = df["agreement_total"]/df["acc_count"]
    df['ip_cnt_per_acc_count'] = df["ip_cnt"]/df["acc_count"]
    df['service1_amt_per_using_time'] = df["service1_amt"]/df["using_time"]
    df["service_cnt_per_time"]=(df["service1_cnt"]+df["service2_cnt"])/df["using_time"]
    
    
    df['op_cnt_per_usign_time'] = (df["op1_cnt"]+df["op2_cnt"])/df["using_time"]
    df['op_cnt_per_login_cnt_avg'] = (df["op1_cnt"]+df["op2_cnt"])/df['login_cnt_avg']
    df['op_cnt_per_login_cnt_p1'] =  (df["op1_cnt"]+df["op2_cnt"])/df['login_cnt_period1']
    df['op_cnt_per_login_cnt_p2'] =  (df["op1_cnt"]+df["op2_cnt"])/df['login_cnt_period2']
    
    # trans
    df = df.merge(gen_user_status_features(trans,'user','amount'), on=['user'], how='left')
    df = df.merge(gen_user_status_features(trans,'user','days_diff'), on=['user'], how='left')
    df = df.merge(gen_user_status_features(trans,'user','hour'), on=['user'], how='left')

  
    print("trans nunique && null features")
    for col in tqdm(['days_diff', 'platform', 'tunnel_in', 'tunnel_out', 'type1', 'type2','ip', 'ip_3']):
        df = df.merge(gen_user_nunique_features(df=trans, value=col, prefix='trans'), on=['user'], how='left')
        df = df.merge(gen_user_null_features(df=trans, value=col, prefix='trans'), on=['user'], how='left')
    df['user_amount_per_days'] = df['user_amount_sum'].div(df['user_trans_days_diff_nuniq'])
    df['user_amount_per_tunnel_in'] = df['user_amount_sum'] / df['user_trans_tunnel_in_nuniq']
    df['user_amount_per_ip_cnt']=df['user_amount_sum']/df['ip_cnt']
    temp=trans[trans['ip'].isnull()].groupby(['user'])['amount'].agg(user_null_ip_amount_sum='sum',
                                                                     user_null_ip_amount_mea='mean'
                                                                    
                                                                    ).reset_index()
    
    df=df.merge(temp,on=['user'],how='left')

            
    print("trans group amount features")
    trans_type1=trans.query("type1==['f67d4b5a05a1352a','19d44f1a51919482','674e8d5860bc033d', '45a1168437c708ff']")
    trans_type2=trans.query("type2==['11a213398ee0c623']")
    trans_plat=trans.query("platform==['46c69cbbce5f1568','42573d7287a8c9c2']")
    trans_tunnel_in=trans.query("tunnel_in==['b2e7fa260df4998d','9c5701005a2d85bd']")
    df = df.merge(gen_user_group_features(df=trans_type2,col='type2',      value='amount'),on=['user'],how='left')
    df = df.merge(gen_user_group_features(df=trans_type1,col='type1',      value='amount'),on=['user'],how='left')
    df = df.merge(gen_user_group_features(df=trans_tunnel_in,col='tunnel_in',  value='amount'),on=['user'],how='left')
    df = df.merge(gen_user_group_features(df=trans,col='tunnel_out',value='amount'), on=['user'], how='left')
    df = df.merge(gen_user_group_features(df=trans_plat,col='platform',   value='amount'),on=['user'],how='left')
    
    print("trans group days_diff features")
    df = df.merge(gen_user_group_features(df=trans_type2,col='type2',      value='days_diff'),on=['user'],how='left')
    df = df.merge(gen_user_group_features(df=trans_type1,col='type1',      value='days_diff'),on=['user'],how='left')
    df = df.merge(gen_user_group_features(df=trans_tunnel_in,col='tunnel_in',  value='days_diff'),on=['user'],how='left')
    df = df.merge(gen_user_group_features(df=trans,col='tunnel_out',value='days_diff'), on=['user'], how='left')
    df = df.merge(gen_user_group_features(df=trans_plat,col='platform',   value='days_diff'),on=['user'],how='left')
    
    
    print("trans group nunique features")
    df = df.merge(gen_user_group_nunique_features(df=trans,col='platform',   value='type1'),on=['user'],how='left')
    df = df.merge(gen_user_group_nunique_features(df=trans,col='platform',   value='ip'),on=['user'],how='left')
    
    print("trans days window features")
    for window in tqdm([20,13,15]):
        df = df.merge(gen_user_window_amount_features(df=trans, window=window), on=['user'], how='left')
        
    print("trans hour window features")
    for win_hour in tqdm([9,18,19,20]):
        df = df.merge(gen_user_window_hour_amount_features(df=trans, window=win_hour), on=['user'], how='left')
        
    print("trans tfidf--count features")    
    df = df.merge(gen_user_tfidf_features(df=trans, value='type1'), on=['user'], how='left')
    df = df.merge(gen_user_tfidf_features(df=trans, value='type2'), on=['user'], how='left')
    
    print("trans LSI features")
    #for col in ['ip','platform','tunnel_in','tunnel_out','ip_3','type1','type2']:
    #    df=df.merge(gen_user_lda_features(trans,col,5),on=['user'],how='left')
        
    #df=df.merge(fasttext(trans,['ip','type1',"type2",'tunnel_out','tunnel_in'],"train"),on=['user'],how='left')
    # op
    print("op nuique features")
    for col in tqdm(['op_type','op_mode','channel']):
        df = df.merge(gen_user_nunique_features(df=op, value=col, prefix='op'), on=['user'], how='left')
    
    print("op null features")
    for col in tqdm(['op_device', 'ip', 'net_type', 'channel', 'ip_3']):
        df = df.merge(gen_user_null_features(df=op, value=col, prefix='op'), on=['user'], how='left')
    
    print("op days_diff features")
    df = df.merge(gen_user_status_features(op,'user','days_diff'), on=['user'], how='left')
    df = df.merge(gen_user_status_features(op,'user','hour'), on=['user'], how='left')

    print("op tfidf--cnt features")
    df = df.merge(gen_user_tfidf_features(df=op, value='op_mode'), on=['user'], how='left')
    df = df.merge(gen_user_tfidf_features(df=op, value='op_type'), on=['user'], how='left')
    df = df.merge(gen_user_tfidf_features(df=op, value='op_device'), on=['user'], how='left')
    df = df.merge(gen_user_countvec_features(df=op, value='op_mode'), on=['user'], how='left')
    df = df.merge(gen_user_countvec_features(df=op, value='op_type'), on=['user'], how='left')
    df = df.merge(gen_user_countvec_features(df=op, value='op_device'), on=['user'], how='left')
    
    print("op LSI features")
    #for col in ["op_device","ip","channel","net_type","op_type","op_mode"]:
    #    df=df.merge(gen_user_lda_features(op,col,5),on=['user'],how='left')
    #df=df.merge(fasttext(op,['op_mode','op_type',"op_device","channel",'ip_3',],"train"),on=['user'],how='left')
    return df
def kfold_stats_feature(train, test, feats, k):
    folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)  # 这里最好和后面模型的K折交叉验证保持一致

    train['fold'] = None
    for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['label'])):
        train.loc[val_idx, 'fold'] = fold_

    kfold_features = []
    for feat in feats:
        nums_columns = ['label']
        for f in nums_columns:
            colname = feat + '_' + f + '_kfold_mean'
            kfold_features.append(colname)
            train[colname] = None
            for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['label'])):
                tmp_trn = train.iloc[trn_idx]
                order_label = tmp_trn.groupby([feat])[f].mean()
                tmp = train.loc[train.fold == fold_, [feat]]
                train.loc[train.fold == fold_, colname] = tmp[feat].map(order_label)
                # fillna
                global_mean = train[f].mean()
                train.loc[train.fold == fold_, colname] = train.loc[train.fold == fold_, colname].fillna(global_mean)
            train[colname] = train[colname].astype(float)

        for f in nums_columns:
            colname = feat + '_' + f + '_kfold_mean'
            test[colname] = None
            order_label = train.groupby([feat])[f].mean()
            test[colname] = test[feat].map(order_label)
            # fillna
            global_mean = train[f].mean()
            test[colname] = test[colname].fillna(global_mean)
            test[colname] = test[colname].astype(float)
    del train['fold']
    return train, test

def lgb_model_origin(train, target, test, k):
    feats = [f for f in train.columns if f not in ['user', 'label']]
    print('Current num of features:', len(feats))
    folds = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)
    oof_probs = np.zeros(train.shape[0])
    output_preds = 0
    offline_score = []
    valid_score = []
    train_score = []
    feature_importance_df = pd.DataFrame()
    parameters = {
        'learning_rate': 0.05,
        'boosting_type': 'gbdt',
        'objective': 'binary',
        'metric': 'auc',
        'num_leaves': 63,
        'bagging_fraction': 0.9,
        'bagging_seed':2020,
        'min_data_in_leaf': 50,
        
        'tree_learner':'voting',
        'reg_alpha':10,
        'reg_lambda':8,
        #'device':'gpu',
        'verbose': -1,
        'nthread': 8,
        'colsample_bytree':0.77,
        'min_child_weight':4,
        'min_child_samples':10,
        'min_split_gain':0,
        'lambda_l1': 0.8,
        'max_bin':300
    }

    for i, (train_index, test_index) in enumerate(folds.split(train, target)):
        train_y, test_y = target[train_index], target[test_index]
        train_X, test_X = train[feats].iloc[train_index, :], train[feats].iloc[test_index, :]
        dtrain = lgb.Dataset(train_X,
                             label=train_y)
        dval = lgb.Dataset(test_X,
                           label=test_y)
        lgb_model = lgb.train(
                parameters,
                dtrain,
                num_boost_round=500,
                valid_sets=[dval],
                early_stopping_rounds=100,
                verbose_eval=100
        )
        oof_probs[test_index] = lgb_model.predict(test_X[feats], num_iteration=lgb_model.best_iteration)
        output_preds += lgb_model.predict(test[feats], num_iteration=lgb_model.best_iteration)/folds.n_splits
        
        offline_score.append(lgb_model.best_score['valid_0']['auc'])
        print(offline_score)
        
        # feature importance
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = lgb_model.feature_importance(importance_type='gain')
        fold_importance_df["fold"] = i + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)

    valid_auc = roc_auc_score(target, oof_probs)
    print('OOF-MEAN-AUC:%.6f, OOF-STD-AUC:%.6f' % (np.mean(offline_score), np.std(offline_score)))
    print("valid_auc:  {}".format(valid_auc))
    print('feature importance:')
    print(feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False).head(40))

    return output_preds, oof_probs, np.mean(offline_score),feature_importance_df.groupby(['feature'])['importance'].mean().sort_values(ascending=False)
# This way we have randomness and are able to reproduce the behaviour within this cell.
          

CPU times: user 20 µs, sys: 0 ns, total: 20 µs
Wall time: 24.6 µs


# 读取数据

In [4]:
%%time
DATA_PATH = '/kaggle/input/financial-risk/'
print('读取数据...')
base_df, op_df, trans_df = data_preprocess(DATA_PATH=DATA_PATH)


读取数据...
CPU times: user 57.4 s, sys: 3.66 s, total: 1min 1s
Wall time: 1min 1s


# 特征工程

In [5]:
%%time
print('开始特征工程...')
data = gen_features(base_df, op_df, trans_df)
data.to_csv('data.csv')

开始特征工程...
base logistic features


  0%|          | 0/8 [00:00<?, ?it/s]

trans nunique && null features


100%|██████████| 8/8 [00:07<00:00,  1.11it/s]


trans group amount features
trans group days_diff features
trans group nunique features


...

trans LSI features
op nuique features


  0%|          | 0/5 [00:00<?, ?it/s]

op null features


100%|██████████| 5/5 [00:06<00:00,  1.26s/it]


op days_diff features
op tfidf--cnt features
op LSI features
CPU times: user 3min 8s, sys: 3.12 s, total: 3min 11s
Wall time: 3min 6s


In [65]:
data_=data

# UID特征

In [56]:
## trans_df uid 查找函数
#uid_list=[]
#l=list(trans_df.columns)
#l.remove('user')
#for i in range(len(l)):
#    for j in range(i+1,len(l)):
#        temp=pd.DataFrame()
#        name=l[i]+'_'+l[j]
#        temp[name]=trans_df[l[i]].astype(str)+trans_df[l[j]].astype(str)
#        if(temp[name].nunique()<10000 and temp[name].nunique()>6):
#            uid_list.append(name)
#            print("{}_{}:  {}".format(l[i],l[j],temp[name].nunique()))
#            
#            trans_temp ,uids= encode_CB(trans_df,[l[i]],[l[j]])
#            uids_FE=[uid+"_FE" for uid in uids]
#            for uid_FE in uids_FE:
#                data_=data_.merge(gen_user_status_features(trans_temp,'user',uid_FE),on=['user'],how='left')

In [66]:
%%time
print("UID trans")
col1_list=['platform']
col2_list=['ip','type2']
trans_temp ,uids= encode_CB(trans_df,col1_list,col2_list)
uids_FE=[uid+"_FE" for uid in uids]
data_=data_.merge(gen_user_status_features(trans_temp,'user','platform_ip_FE',['mean','sum']),on=['user'],how='left')
data_=data_.merge(gen_user_status_features(trans_temp,'user','platform_type2_FE',['sum','min']),on=['user'],how='left')


col1_list=['type1']
col2_list=['hour']
trans_temp ,uids= encode_CB(trans_df,col1_list,col2_list)
uid_FE=[uid+"_FE" for uid in uids]
data_=data_.merge(gen_user_status_features(trans_temp,'user',uids_FE[0],['mean']),on=['user'],how='left')
#data_=data_.merge(gen_user_status_features(trans_temp,'user',uids_FE[1],['mean','std']),on=['user'],how='left')



col1_list=['type2']
col2_list=['hour']
trans_temp ,uids= encode_CB(trans_df,col1_list,col2_list)
uid_FE=[uid+"_FE" for uid in uids]
data_=data_.merge(gen_user_status_features(trans_temp,'user',uids_FE[0],['sum']),on=['user'],how='left')
#data_=data_.merge(gen_user_status_features(trans_temp,'user',uids_FE[1],['mean','sum','CV']),on=['user'],how='left')


col1_list=['days_diff']
col2_list=['hour','week']
trans_temp ,uids= encode_CB(trans_df,col1_list,col2_list)
uid_FE=[uid+"_FE" for uid in uids]
data_=data_.merge(gen_user_status_features(trans_temp,'user',uids_FE[0],['mean','max']),on=['user'],how='left')
data_=data_.merge(gen_user_status_features(trans_temp,'user',uids_FE[1],['sum']),on=['user'],how='left')
#data_=data_.merge(gen_user_status_features(trans_temp,'user',uids_FE[2],['mean']),on=['user'],how='left')




UID trans
CPU times: user 9.24 s, sys: 575 ms, total: 9.81 s
Wall time: 9.81 s


In [58]:
## trans_df uid 查找函数
#
#uid_list=[]
#def find_uid(base_df):
#     l=list(base_df.columns)
#     l.remove('user')
#     l.remove('label')
#    
#     for i in range(len(l)):
#         for j in range(i+1,len(l)):
#            if(base_df[l[i]].dtype=='object' and base_df[l[j]].dtype=='object' and
#              'card_' not in l[i] and 'card_' not in l[j] and 'agreement' not in l[i] 
#               and 'agreement' not in l[j] and 
#               '_amount' not in l[i] and '_amount' not in  l[j] ):
#                 temp=pd.DataFrame()
#                 name=l[i]+'_'+l[j]
#                 temp[name]=base_df[l[i]].astype(str)+base_df[l[j]].astype(str)
#                 if(temp[name].nunique()<500 and temp[name].nunique()>20):
#                     uid_list.append(name)
#                     print("{}_{}:  {}".format(l[i],l[j],temp[name].nunique()))
#                     data_[name]=data_[l[i]]+data_[l[j]]
#     print(len(uid_list))
#drop_list+=uid_list
#find_uid(base_df)
#
## trans_df uid 查找函数
   

In [59]:
#col1_list=['provider']
#col2_list=['level', 'verified',
#       'using_time', 'regist_type','op1_cnt', 'op2_cnt','province', 'city', 'balance', 'balance_avg', 'balance1',
#       'balance1_avg', 'balance2', 'balance2_avg', 'service3']
#data_ ,uid_list= encode_CB(data_,col1_list,col2_list)
#drop_list+=uid_list
#u=[
#'user_trans_ip_count',
#'user_days_diff_sum_x',
#'product7_fail_ratio',
#'user_trans_ip_3_count',
#'product7_fail_cnt',
#'login_cnt_period_CV',
#'svd_countvec_op_mode_4',
#'svd_tfidf_type1_1'
#]
#
#
#
#for i in u:
#    for j in tqdm():
#        data_=data_.merge(gen_user_status_features(data_,j,i),on=[j],how='left')

In [75]:
%%time
print("UID BASE")
data_['uid1']=data_['agreement1']+data_['agreement2']+data_['agreement3']+data_['agreement4']
drop_list+=['uid1']
data_=data_.merge(gen_user_status_features(data_,'uid1','user_trans_ip_count',['std']),on=['uid1'],how='left')          

UID BASE
CPU times: user 187 ms, sys: 9.87 ms, total: 197 ms
Wall time: 195 ms


In [83]:
zip_list=[
            [['provider'],['using_time'],['user_amount_CV','svd_countvec_op_mode_4'],['mean']],
          [['provider'],['province'],['product7_fail_cnt'],['mean']],
          [['provider'],['service3'],['user_amount_sum'],['sum']]
         ]

for i in zip_list:
    data_ ,uids= encode_CB(data_,i[0],i[1])
    uid_FE=[uid+"_FE" for uid in uids]
    drop_list+=uid_FE
    drop_list+=uids
    for j in i[2]:
        data_=data_.merge(gen_user_status_features(data_,uids[0],j,i[3]),on=[uids[0]],how='left') 

In [91]:
for i in drop_list:
    if i not in data_.columns:
        print(i)
        drop_list.remove(i)

provider_verified
provider_balance2
provider_op2_cnt
provider_balance_avg


In [92]:
drop_list=list(set(drop_list))
drop_list

['provider_province_FE',
 'provider_using_time',
 'uid1',
 'provider_province',
 'provider_using_time_FE',
 'provider_service3_FE',
 'provider_service3']

In [93]:
%%time
print('开始模型训练...')
k_fold=10

train = data_[~data_['label'].isnull()].copy()
test = data_[data_['label'].isnull()].copy()
target = train['label']
#train,test = encode_Cat(train,test,k=k_fold)
#train, test = kfold_stats_feature(train, test, uids_2d, k_fold)

for i in ['service3_level']+drop_list:    
    train.drop([i], axis=1, inplace=True)
    test.drop([i], axis=1, inplace=True)
        

train,test=encode_LE(train,test)

lgb_preds, lgb_oof, lgb_score,feature_importance_df = lgb_model_origin(train=train, target=target, test=test, k=k_fold)

sub_df = test[['user']].copy()
sub_df['prob'] = lgb_preds
sub_df.to_csv('sub(10folds).csv', index=False)
feature_importance_df.to_csv('feature(1).csv')

开始模型训练...


100%|██████████| 24/24 [00:01<00:00, 14.05it/s]


Current num of features: 463
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.740916
[200]	valid_0's auc: 0.741953
Early stopping, best iteration is:
[190]	valid_0's auc: 0.742598
[0.7425982384544308]
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.735165
[200]	valid_0's auc: 0.736412
Early stopping, best iteration is:
[159]	valid_0's auc: 0.736848
[0.7425982384544308, 0.7368482353304543]
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.726977
[200]	valid_0's auc: 0.72805
Early stopping, best iteration is:
[146]	valid_0's auc: 0.728352
[0.7425982384544308, 0.7368482353304543, 0.7283524924542539]
Training until validation scores don't improve for 100 rounds
[100]	valid_0's auc: 0.730454
[200]	valid_0's auc: 0.731726
Early stopping, best iteration is:
[179]	valid_0's auc: 0.732291
[0.7425982384544308, 0.7368482353304543, 0.7283524924542539, 0.7322909592529712]
Training until valid

In [94]:
f=pd.DataFrame(feature_importance_df,columns=['importance']).reset_index()

In [101]:
good_uid=[]
for i in drop_list:
    for j in f.feature:
        if((i in j )):
            if (f[f.feature==j].importance.values > 1000):
                good_uid.append(j)
                print("{}      :     {}".format(i,j))
print(len(good_uid))

provider_using_time      :     provider_using_time_svd_countvec_op_mode_4_mean
uid1      :     uid1_user_trans_ip_count_std_x
provider_province      :     provider_province_product7_fail_cnt_mean
3


In [53]:
good_uid=[]
for i in uid_list:
    for j in f.feature:
        if( i+"_FE" in j ):
            if (f[f.feature==j].importance.values > 100):
                good_uid.append(j)
                print("{}:{}:      {}".format(i,j,f[f.feature==j].importance.values))
print(len(good_uid))

provider_using_time:provider_using_time_FE:      [170.32847862]
1


# --------------分界线--------------

In [ ]:
def w2v_feat(df, feat, mode):
    data_frame=df.copy()
    for i in feat:
        if data_frame[i].dtype != 'object':
            data_frame[i] = data_frame[i].astype(str)
    data_frame.fillna('nan', inplace=True)

    print(f'Start {mode} word2vec ...')
    model = Word2Vec(data_frame[feat].values.tolist(), size=10, window=2, min_count=1,
                     workers=multiprocessing.cpu_count(), iter=10)
    stat_list = ['min', 'max', 'mean', 'std']
    new_all = pd.DataFrame()
    for m, t in enumerate(feat):
        print(f'Start gen feat of {t} ...')
        tmp = []
        for i in data_frame[t].unique():
            tmp_v = [i]
            tmp_v.extend(model[i])
            tmp.append(tmp_v)
        tmp_df = pd.DataFrame(tmp)
        w2c_list = [f'w2c_{t}_{n}' for n in range(10)]
        tmp_df.columns = [t] + w2c_list
        tmp_df = data_frame[['user', t]].merge(tmp_df, on=t)
        tmp_df = tmp_df.drop_duplicates().groupby('user').agg(stat_list).reset_index()
        tmp_df.columns = ['user'] + [f'{p}_{q}' for p in w2c_list for q in stat_list]
        if m == 0:
            new_all = pd.concat([new_all, tmp_df], axis=1)
        else:
            new_all = pd.merge(new_all, tmp_df, how='left', on='user')
    return new_all
def gen_user_lda_features(df, value,num_topics):
    df[value] = df[value].astype(str)
    df[value].fillna('-1', inplace=True)
    group_df = df.groupby(['user']).apply(lambda x: x[value].tolist()).reset_index()
    group_df.columns = ['user', 'list']
    
    texts=group_df['list']
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
#    tfidf = models.TfidfModel(corpus)
#   corpus_tfidf = tfidf[corpus]
    lda_model = models.LdaMulticore(corpus, id2word=dictionary, num_topics=num_topics,workers=multiprocessing.cpu_count(),random_state=2020)
    corpus_lda = lda_model.get_document_topics(corpus,minimum_probability=0)
    
    new_all = pd.DataFrame(corpus_lda)
    new_all.columns = ["LDA_{}_{}".format(value,i) for i in range(num_topics)]
    for col in tqdm(new_all.columns):
        try:
           new_all[col]=new_all[col].apply(lambda x:x[1])
        except:
            None
             
    new_all['user']=group_df['user']
    
    return new_all
def fasttext(df, feat, mode):
    data_frame=df.copy()
    # 转化为str
    for i in feat:
        if data_frame[i].dtype != 'object':
            data_frame[i] = data_frame[i].astype(str)
    data_frame.fillna('nan', inplace=True) # 甜宠

    print(f'Start {mode} FastText ...')
    model = FastText(data_frame[feat].values.tolist(),  size=10, window=3, min_count=1, 
                     workers=multiprocessing.cpu_count(), iter=10,min_n = 3 , max_n = 6,word_ngrams = 0)
    stat_list = ['min', 'max', 'mean', 'std']
    new_all = pd.DataFrame()
    for m, t in enumerate(feat):
        print(f'Start gen feat of {t} ...')
        tmp = []
        for i in data_frame[t].unique():
            tmp_v = [i]
            tmp_v.extend(model[i])
            tmp.append(tmp_v)
        tmp_df = pd.DataFrame(tmp)
        w2c_list = [f'fast_text_{t}_{n}' for n in range(10)]
        tmp_df.columns = [t] + w2c_list
        tmp_df = data_frame[['user', t]].merge(tmp_df, on=t)
        tmp_df = tmp_df.drop_duplicates().groupby('user').agg(stat_list).reset_index()
        tmp_df.columns = ['user'] + [f'{p}_{q}' for p in w2c_list for q in stat_list]
        if m == 0:
            new_all = pd.concat([new_all, tmp_df], axis=1)
        else:
            new_all = pd.merge(new_all, tmp_df, how='left', on='user')
    return new_all




# ---分界线---

## 测试target encoder是否有效

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from gensim import corpora
from gensim import models
DATA_PATH = '/kaggle/input/financial-risk/'
train_trans = pd.read_csv(DATA_PATH+'train_trans.csv')
test_trans = pd.read_csv(DATA_PATH+'test_a_trans.csv')
train_base = pd.read_csv(DATA_PATH+'train_base.csv')
train_label = pd.read_csv(DATA_PATH+'train_label.csv')
train_base = train_base.merge(train_label,on=['user'],how='left')
test_base= pd.read_csv(DATA_PATH+  'test_a_base.csv')
targer = pd.read_csv(DATA_PATH+'train_label.csv')

train_op = pd.read_csv(DATA_PATH+'train_op.csv')
test_op = pd.read_csv(DATA_PATH+'test_a_op.csv')

In [ ]:
col1_list=['platform']
col2_list=['ip','type1','type2','tunnel_in','tunnel_out']
trans_temp ,uids= encode_CB(train_trans,col1_list,col2_list)
#for index in uids:
#    df = df.merge(gen_user_status_features(trans_temp,index,'amount'), on=['user'], how='left')
#    drop_list.append(uids)
            

In [ ]:
["LDS_{}_{}".format('ip',i) for i in range(10)]

In [ ]:

train_base['city_balance_avg'] = train_base['city'].map(str) + '_' + train_base['balance_avg'].map(str)
train_base['city_level'] = train_base['city'].map(str) + '_' + train_base['level'].map(str)
train_base['city_product1_amount'] = train_base['city'].map(str) + '_' + train_base['product1_amount'].map(str)
train_base['city_balance1_avg'] = train_base['city'].map(str) + '_' + train_base['balance1_avg'].map(str)
train_base['city_balance2_avg'] = train_base['city'].map(str) + '_' + train_base['balance2_avg'].map(str)

test_base['city_balance_avg'] =  test_base['city'].map(str) + '_' + test_base['balance_avg'].map(str)
test_base['city_level'] =        test_base['city'].map(str) + '_' + test_base['level'].map(str)
test_base['city_product1_amount'] = test_base['city'].map(str) + '_' + test_base['product1_amount'].map(str)
test_base['city_balance1_avg'] = test_base['city'].map(str) + '_' + test_base['balance1_avg'].map(str)
test_base['city_balance2_avg'] = test_base['city'].map(str) + '_' + test_base['balance2_avg'].map(str)


### 自己定义的target encoder

In [ ]:
def encode_Cat_1(train,test, feature, k,noise_level=0.01,smoothing=20):
    
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)    

    for col in feature:
        train['target_'+col] = 0
        test['target_' + col] =0
        if col not in orgin_features:
            smoothing=50
            noise_level=0.7
        for trn_idx,val_idx in kf.split(train,train['label']):
            X = train.iloc[trn_idx][col]
            enc_tar = TargetEncoder(smoothing=smoothing).fit(X,train.iloc[trn_idx]['label'])
            train['target_'+col] += add_noise(enc_tar.transform(train[col])[col],noise_level) / k
            test['target_'+col] += add_noise(enc_tar.transform(test[col])[col],noise_level) / k
       
    return  train,test



def Catboost_encoding(train,test, feature, k,noise_level=0.01,sigma=2.0,a=1):
    
    kf = StratifiedKFold(n_splits=k, shuffle=True, random_state=2020)    

    for col in feature:
        train['catboost_'+col] = 0
        test['catboost_' + col] =0
        if col not in orgin_features:
            sigma=5.0
            a = 2
        for trn_idx,val_idx in kf.split(train,train['label']):
            X = train.iloc[trn_idx][col]
            enc_tar = CatBoostEncoder(sigma=sigma,a=a).fit(X,train.iloc[trn_idx]['label'])
            train['catboost_'+col] += add_noise(enc_tar.transform(train[col])[col],noise_level) / k
            test['catboost_'+col] += add_noise(enc_tar.transform(test[col])[col],noise_level) / k        
    return  train,test

In [ ]:
# 评价编码函数
folds = StratifiedKFold(n_splits=10, shuffle=True, random_state=2020)
f_cats = ['province','city',"city_level",'city_balance1_avg','city_balance2_avg',
                      "city_product1_amount",'city_balance_avg']

print("%20s   %20s | %20s | %20s  | %20s" % ("", "Raw Categories","Encoded Categories","Baseline Categories","CatBoost Categories"))
for f in f_cats:
    print("%-20s : " % f, end="")
    origin_scores = []
    baseline_scores = []
    target_scores=[]   
    catboost_scores=[]
    for trn_idx, val_idx in folds.split(train_base, train_base.label):
        
        val_f = train_base.iloc[trn_idx].groupby([f])['label'].transform('mean')
        val_f.name=f
        val_tgt = train_base.label.iloc[trn_idx]
        
        trn_f = train_base.iloc[trn_idx][f]
        trn_tgt = train_base.label.iloc[trn_idx]
        
        val_tf ,temp = encode_Cat_1(train_base.iloc[trn_idx],test_base,[f],5)
        val_tf1 ,temp = kfold_stats_feature(train_base.iloc[trn_idx].reset_index(),test_base,[f],5)  
        val_tf2 ,temp = Catboost_encoding(train_base.iloc[trn_idx],test_base,[f],5)
        
        origin_scores.append(max(roc_auc_score(val_tgt, val_f), 1 - roc_auc_score(val_tgt, val_f)))
        target_scores.append(roc_auc_score(val_tgt, val_tf['target_'+f]))
        baseline_scores.append(roc_auc_score(val_tgt,val_tf1[f+'_label_kfold_mean']))
        catboost_scores.append(roc_auc_score(val_tgt, val_tf2['catboost_'+f]))
    val_tf1 ,temp = kfold_stats_feature(train_base,test_base,[f],5)    
    print(" %.6f + %.6f | %6f + %.6f  | %.6f + %.6f  | %.6f + %.6f" 
          % (np.mean( origin_scores), np.std( origin_scores), np.mean(target_scores), np.std(target_scores),
             np.mean(baseline_scores), np.std(baseline_scores),np.mean(catboost_scores), np.std(catboost_scores)))

In [ ]:
#  自己写target encoder的评价方法,没有采用5折方法
def encode_val(train_base,test_base,cols):
    for col in cols:
    
        temp_train,temp_test=kfold_stats_feature(train_base,test_base,[col],5)
        temp=train_base.groupby([col])['label'].agg(cnt='count',mean='mean')
        temp=temp.reset_index()
        temp_train=temp_train.merge(temp,on=[col],how='left')
        
        
        print("BASELINE Encode score: {}".format(roc_auc_score(temp_train['label'],temp_train['{}_label_kfold_mean'.format(col)])))
        print("Myown Encode score:    {}".format(roc_auc_score(temp_train['label'],temp_train['Mean_encoded_{}'.format(col)])))
        print("Origin score:          {}".format(roc_auc_score(temp_train['label'],temp_train['mean'])))
        print()
cols = ['province','city',"city_level",'city_balance1_avg','city_balance2_avg',
                      "city_product1_amount",'city_balance_avg']
encode_val(train_base,test_base,cols)